In [ ]:
import sys

# TODO: Append the path to your `multi-annotator-machine-learning` project.
sys.path.append("../../")

from skactiveml.utils import ExtLabelEncoder
from maml.data import Dopanim
from maml.utils import (
    plot_annotation_calibration_curves,
    plot_annotation_times_histograms,
    plot_annotation_confusion_matrices,
    plot_annotator_metadata_correlation,
)

# TODO: Adjust data path to your dataset.
DATA_PATH = "."

# TODO: Adjust flag for downloading the dataset.
DOWNLOAD = False

# TODO: Adjust path for saving figures.
FIGURE_PATH = "."

### Load Data

In [ ]:
train_ds = Dopanim(DATA_PATH, version='train', variant='full', download=DOWNLOAD)
y_true, observation_ids = train_ds.load_true_class_labels()
le = ExtLabelEncoder(missing_label=None).fit(y_true)
y_true_enc = le.transform(y_true)
annotation_inconsistencies = train_ds.load_annotation_consistencies(
    observation_ids=observation_ids, annotators=train_ds.annotators
)
annotation_times = train_ds.load_annotation_times(
    observation_ids=observation_ids,
    annotators=train_ds.annotators,
)
likelihoods = train_ds.load_likelihoods(
    observation_ids=observation_ids,
    annotators=train_ds.annotators,
    classes=le.classes_,
    normalize=True,
)
annotator_metadata, annotator_feature_names = train_ds.load_annotator_metadata(
    annotators=train_ds.annotators, classes=train_ds.classes
)

### Plot Calibration Curves

In [ ]:
plot_annotation_calibration_curves(
    likelihoods=likelihoods, y_true=y_true, classes=train_ds.classes, n_bins=10, path=FIGURE_PATH,
)

### Plot Annotation Times

In [ ]:
plot_annotation_times_histograms(times=annotation_times, n_bins=50, path=FIGURE_PATH)

### Plot Confusion Matrices

In [ ]:
cm_all = plot_annotation_confusion_matrices(
    likelihoods=likelihoods,
    classes=train_ds.classes,
    y_true=y_true,
    path=FIGURE_PATH,
)

In [ ]:
print(annotator_feature_names)
sub_indices = [12, 13, 4, 3, 9, 0]
annotator_feature_names_sub = annotator_feature_names[sub_indices]
annotator_metadata_sub = annotator_metadata[:, sub_indices]
plot_annotator_metadata_correlation(
    annotator_metadata=annotator_metadata_sub,
    annotator_feature_names=annotator_feature_names_sub,
    cm_all=cm_all,
    path=FIGURE_PATH,
)